In [1]:
import torch
import numpy as np
from transformers.pytorch_utils import Conv1D

from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

/home/jeremy/miniconda3/envs/gan/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
tokenizer_file = "./tokenizers/tinystories.json"
model_dir = "./models/8b0d1f8f9825427ab8e0321c05ddd0a9/6400"

In [25]:
tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_file)
tokenizer.bos_token = "<|endoftext|>"
tokenizer.eos_token = "<|endoftext|>"

model = GPT2LMHeadModel.from_pretrained(model_dir, local_files_only=True)

In [4]:
trainable_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(trainable_parameters)

total_parameters = sum(p.numel() for p in model.parameters())
print(total_parameters)

1105536
1105536


In [5]:
model.generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "eos_token_id": 0,
  "transformers_version": "4.32.1"
}

In [26]:
text_input = "Once upon a time,"

In [31]:
token_input = tokenizer(text_input, return_tensors="pt")
token_output = model.generate(**token_input, top_p=0.9, max_length=80, temperature=1.0)
text_output = tokenizer.batch_decode(token_output, skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [30]:
text_output

[' Once upon a time, there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there there']

In [22]:
for mn, m in model.named_modules():
    print(mn)


transformer
transformer.wte
transformer.wpe
transformer.drop
transformer.h
transformer.h.0
transformer.h.0.ln_1
transformer.h.0.attn
transformer.h.0.attn.c_attn
transformer.h.0.attn.c_proj
transformer.h.0.attn.attn_dropout
transformer.h.0.attn.resid_dropout
transformer.h.0.ln_2
transformer.h.0.mlp
transformer.h.0.mlp.c_fc
transformer.h.0.mlp.c_proj
transformer.h.0.mlp.act
transformer.h.0.mlp.dropout
transformer.h.1
transformer.h.1.ln_1
transformer.h.1.attn
transformer.h.1.attn.c_attn
transformer.h.1.attn.c_proj
transformer.h.1.attn.attn_dropout
transformer.h.1.attn.resid_dropout
transformer.h.1.ln_2
transformer.h.1.mlp
transformer.h.1.mlp.c_fc
transformer.h.1.mlp.c_proj
transformer.h.1.mlp.act
transformer.h.1.mlp.dropout
transformer.h.2
transformer.h.2.ln_1
transformer.h.2.attn
transformer.h.2.attn.c_attn
transformer.h.2.attn.c_proj
transformer.h.2.attn.attn_dropout
transformer.h.2.attn.resid_dropout
transformer.h.2.ln_2
transformer.h.2.mlp
transformer.h.2.mlp.c_fc
transformer.h.2.mlp